# 1 Importa as bibliotecas

In [ ]:
import pandas as pd
from datetime import datetime

# 2 Lê os arquivos

In [ ]:
## Para uso no Kaggle Notebook
path = '../input/walmart-recruiting-store-sales-forecasting/'

## Para uso local
#path = 'data/'

train = pd.read_csv(path + 'train.csv.zip')
test = pd.read_csv(path + 'test.csv.zip')
stores = pd.read_csv(path + 'stores.csv')
features = pd.read_csv(path + 'features.csv.zip')
sampleSubmission = pd.read_csv(path + 'sampleSubmission.csv.zip')

# 3 Análise dos dados

# 3.1 Stores

In [ ]:
print('Shape', stores.shape)
print('\nTypes\n', stores.dtypes)
print('\nHead\n', stores.head())
print('\nTipos de lojas:')
print(stores['Type'].value_counts())
print('\nSize:  min:',stores.Size.min(),'  max:', stores.Size.max())
print('\nLista de lojas:', stores.Store.unique())
print('\nQuantidade de lojas:', stores.Store.nunique())
sizes = stores.Size
sizes.plot.hist()

# 3.2 Train

In [ ]:
print('Shape', train.shape)
print('\nTypes\n', train.dtypes)
print('\nHead\n', train.head())
print('\nHead\n', train.tail())

In [ ]:
# Lista de lojas e departamentos
print('\nLista de lojas:', train.Store.unique())
print('\nQuantidade de lojas:', train.Store.nunique())
print('\nLista de departamentos:', sorted(train.Dept.unique()))
print('\nQuantidade de departamentos:', train.Dept.nunique())

In [ ]:
#Cria um índice Loja-Departamento
train['StoreDept'] = train['Store'].astype(str) + '-' + train['Dept'].astype(str) 
print('Combinções Loja-Departamento:', train.StoreDept.nunique())

# Cria um vetor com as combinações loja-departamento
trainStoreDept = train.StoreDept.unique()

#Verifica o tamanho das séries
trainSeriesSize = train.StoreDept.value_counts().reset_index()
trainSeriesSize.columns = ['StoreDept','Count']

trainSeriesSizeGrouped = trainSeriesSize.Count.value_counts().reset_index()
trainSeriesSizeGrouped.columns = ['Size','Count']
print(trainSeriesSizeGrouped)

#Cria um vetor com as combinações de loja-departamento com seéries com 143 semanas
seriesWith143Weeks = trainSeriesSize[trainSeriesSize.Count == 143]
seriesWith143Weeks = seriesWith143Weeks.StoreDept.unique()

#Verifica o tamanho da maior série possível 
maxDate = datetime.strptime(train.Date.max(), '%Y-%m-%d')
minDate=  datetime.strptime(train.Date.min(), '%Y-%m-%d')
maxWeeks = abs((maxDate - minDate).days/7 + 1)
print("Máxima série possível:",maxWeeks)

In [ ]:
# Prepara os dados para plotar o gráfico de vendas
train['year'] = pd.to_datetime(train['Date']).dt.year
train['week'] = pd.to_datetime(train['Date']).dt.isocalendar().week
plotTrain = train.groupby(['year','week'])['Weekly_Sales'].sum().reset_index(name='Weekly_Sales')
plotTrain.pivot(index='week', columns='year', values='Weekly_Sales').plot(legend=True, kind="line", figsize=(20,5), xticks=range(1,53))

In [ ]:
# Semanas com feriado
IsHoliday = train[train.IsHoliday == True]
IsHoliday.groupby(['Date'])['week'].max().reset_index(name='week')

- Todas as lojas nos dados de treino estão na lista de lojas.
- Nem todos os departamentos existem. Por exemplo não existe o departamento 15.
- Os dados de treino possuem 3331 conjuntos Loja-Departamento, dessas 2660 (79,8%) possuem 143 semanas de dados.
- A maior série possível no conjunto de dados de treino é com 143 semanas. 
- São visiveis dois picos de vendas no fim do ano provavelmente referentes aos feriados de ação de graças e natal.
- Os feriados acontecem sempre nas mesmas semanas.

# 3.3 Test

In [ ]:
print('Shape', test.shape)
print('\nTypes\n', test.dtypes)
print('\nHead\n', test.head())
print('\nHead\n', test.tail())

In [ ]:
# Lista de lojas e departamentos
print('\nLista de lojas:', test.Store.unique())
print('\nQuantidade de lojas:', test.Store.nunique())
print('\nLista de departamentos:', sorted(test.Dept.unique()))
print('\nQuantidade de departamentos:', test.Dept.nunique())

In [ ]:
#Cria um índice Loja-Departamento
test['StoreDept'] = test['Store'].astype(str) + '-' + test['Dept'].astype(str) 
print('Combinções Loja-Departamento:', test.StoreDept.nunique())
testStoreDept = test.StoreDept.unique()

In [ ]:
# Verifica se todas as lojas nos dados de teste estão nos dados de treino
print(set(test.Store.unique()) - set(train.Store.unique()))
# Verifica se todas as lojas nos dados de treino estão nos dados de teste
print(set(train.Store.unique()) - set(test.Store.unique()))

In [ ]:
# Verifica se todos os departamentos nos dados de teste estão nos dados de treino
print(set(test.Dept.unique()) - set(train.Dept.unique()))
# Verifica se todos os departamentos nos dados de treino estão nos dados de teste
print(set(train.Dept.unique()) - set(test.Dept.unique()))

In [ ]:
# Verifica se todos os conjuntos loja-departamento nos dados de teste estão nos dados de treino
diff = set(testStoreDept) - set(trainStoreDept)
print(len(diff), 'Conjuntos loja-departamento que estão mos dados de teste não estão nos dados de treino')
print(diff)

# Verifica se todos os conjuntos loja-departamento nos dados de treino estão nos dados de teste
diff2 = set(trainStoreDept) - set(testStoreDept)
print('\n', len(diff2), 'Conjuntos loja-departamento que estão mos dados de treino e não estão nos dados de teste')
print(diff2)

In [ ]:
# Semanas com feriado
IsHoliday = test[test.IsHoliday == True].copy()
IsHoliday['week'] = pd.to_datetime(test['Date']).dt.isocalendar().week
IsHoliday.groupby(['Date'])['week'].max().reset_index(name='week')

- 11 Conjuntos loja-departamento que precisam de forecast não possuem dados de treino.
- Todas lojas dos dados de teste estão nos dados de treino.
- Todos os departamentos dos dados de teste estão nos dados de treino.
- Os feriados dos dados de testes acontecem na mesma semana que os feriados dos dados de treino.

# 3.4 Features

In [ ]:
print('Shape', features.shape)
print('\nTypes\n', features.dtypes)
features.head()

In [ ]:
# Verifica os dados nulos 
features.isna().sum()

In [ ]:
# Verifica se todas as lojas possuem features
print('\nLista de lojas:', features.Store.unique())
print('\nQuantidade de lojas:', features.Store.nunique())

In [ ]:
# Verifica a quantidade de semanas com features para cada loja
featuresSeriesSize = features.Store.value_counts().reset_index()
featuresSeriesSize.columns = ['Store','Count']
print(featuresSeriesSize)

In [ ]:
features.tail()

- Todas as lojas possuem features para 182 semanas.
- Os dados vão da primeira semana dos dados de treino até a última semana dos dados de teste.
- Não há dados de MarkDown para todas as semanas, como já esperado pela descrição dos dados.
- Faltam alguns dados de CPI e Unemployment.
- Além das features do arquivo de features podemos considerar os dados de tamanho e tipo de loja.

# 3.5 Submission

In [ ]:
print('Shape', sampleSubmission.shape)
print('\nTypes\n', sampleSubmission.dtypes)
print('\nHead\n', sampleSubmission.head())
print('\nHead\n', sampleSubmission.tail())

# 4 Proposta de solução

A solução proposta será composta de 3 modelos distintos. Dois modelos puramente de séries temporais e um modelo de regressão linear. O forecast final será definido como descrito a seguir:

- Para os conjuntos de loja-departamento que possuem 143 semanas de dados. O forecast será a média dos forecasts dos 3 modelos propostos.
- Para os demais conjuntos será considerado apenas o modelo de regressão linear. 

Utilizar a média de diferentes modelos é uma das boas práticas apresentadas por [Hyndman, R.J., & Athanasopoulos, G. (2018)](https://otexts.com/fpp2/). Os autores citam em seu livro um trecho escrito por <a href="https://www.sciencedirect.com/science/article/abs/pii/0169207089900125" target="_blank">Clemen (1989) </a>:

`The results have been virtually unanimous: combining multiple forecasts leads to increased forecast accuracy. In many cases one can make dramatic performance improvements by simply averaging the forecasts.`

Ainda em seu livro *Rob J Hyndman e George Athanasopoulos* comentam sobre a dificuldade de trabalhar com séries semanais pois o periodo sazonal (número de semanas no ano) é grande e não inteiro. Nestes casos os autores sugerem o uso de decomposição STL com um método não sazonal aplicado nos dados ajustados sazonalmente. Sendo assim para os modelos de séries temporais serão utilizados os métodos ARiMA e Exponential Smoothing nos dados ajustados sazonalmente.

# 4.1 Implementação dos modelos

Os modelos serão implementados em R utilizando o pacote [forecast](https://pkg.robjhyndman.com/forecast/). Está escolha se dá, pois, conheço o livro dos criadores da biblioteca e já utilizei anteriormente mesmo com conhecimentos bem limitados em R.

# 4.2 Modelos de séries temporais

Os dados serão preparados em python para uso na biblioteca R. 
Serão utilizados os modelos STL + Exponential Smoothing e STL + ARIMA. Para otimização dos parâmetros será utilizado como critério o AICc (Corrected Akaike’s Information Criterion). 

Os modelos de séries temporais estão implementados no notebook [WR-SSF-TimeSeries](https://www.kaggle.com/code/filipeasantos/wr-ssf-timeseries).

In [ ]:
# Prepara os dados para enviar ao R
trainToTimeSeriesModels = train[train['StoreDept'].isin(seriesWith143Weeks)]
trainToTimeSeriesModels = trainToTimeSeriesModels.drop(columns=['Store', 'Dept', 'week', 'IsHoliday', 'year'])
trainToTimeSeriesModels = trainToTimeSeriesModels[['StoreDept','Date', 'Weekly_Sales']]
trainToTimeSeriesModels = pd.pivot_table(trainToTimeSeriesModels, values='Weekly_Sales', index='StoreDept', columns='Date').reset_index()
trainToTimeSeriesModels.to_csv('trainToTimeSeriesModels.csv')

# Salva os dados filtratos em um CSV.

#Kaggle
trainToTimeSeriesModels.to_csv('trainToTimeSeriesModels.csv', index=False)

#Local
#trainToTimeSeriesModels.to_csv('data/trainToTimeSeriesModels.csv', index=False)

In [ ]:
# Lê os resultados dos modelos de séries temporais
## Para uso no Kaggle Notebook
path = '../input/wr-ssf-timeseries/'

## Para uso local
#path = 'data/'

fcstsARIMA = pd.read_csv(path + 'fcstsARIMA.csv').set_index('StoreDept')
fcstsETS = pd.read_csv(path + 'fcstsETS.csv').set_index('StoreDept')

# Verifica os resultados
display(fcstsARIMA)
display(fcstsETS)

Para os modelos de Exponential Smoothing analisando os dados da amostragem que foi plotada dos modelos observa-se que em conforme esperado nenhum dos modelos avaliados foi considerada a caracteristica de sazonalidade, visto que os dados foram ajustados sazonalmente. Observou-se também que em alguns a caracteristica de trend foi considerada e em alguna não. Essa informação sugere que seria interessante comparar no modelo de regressão linear a utilização ou não da trend. 

Para os modelos ARIMA observa-se uma variação na escolha dos parâmetros.

Analisand os gráficos dos modelos de Exponential Smoothing e ARIMA é possivel avaliar que:
- as médias dos residuos são sempre muito próximas de zero,
- no tempo os residuos apresentam aprarentemente uma variação constante em torno de zero mas em alguns casos há pico de variações e para alguns modelos a amplitude é muito grande e esses casos precisam ser avaliados com mais detalhe, 
- as auto-correlações possuem alguns picos mas para todos os modelos mais de 95% dos picos estão dentro do limite acetável. 



# 4.3 Modelo de regessão linear

Para uma versão preliminar do modelo de regressão serão utilizados os seguinte atributos:
- Tamanho da Loja
- Tipo da loja (3)
- Departamento (81)
- Semanas (52)
- Trend

A fim de começar com um modelo com poucos atributos, os dados de Markdowns, CPI e Unemployment não serão utilizados em um primeiro momento pois possuem dados faltantes e é necessário uma análise mais detalhada. Optou-se por não utilizar os dados de Temperature e Fuel_Price no primeiro momento. Após o desenvolvimento do modelo preliminar serão realizadas análises e os atributos escolhidos podem ser removidos e novos atributos poderão ser adicionados. 

Unemployment e Fuel_Price merecem uma atenção detalhada pois os dois podem ser consequências de um outro fator, por exemplo a inflação, e colocar os dois no modelo poderá gerar redundância. 

CPI acredito que deve ser importante para o modelo mas exige uma análise detalhada dos dados faltantes que deixo para ser feita posteriormente ao desenvolvimento de uma primeira versão. 

Para os feriados seria escolhido a utilização de 4 variáveis para representar cada um dos tipos de feriados distintos, pois, analisando o gráfico de vendas ao longo do tempo pode-se perceber que cada feriado causa um efeito diferente nas vendas.

Entretanto também se escolheu que o modelo considerará a semana como um atributo e sendo que os feriados sempre caem nas mesmas semanas optou-se por não utilizar variáveis de feriados para evitar redundância. 


O modelo de regressão linear está implementado no notebook [WR-SSF-LinearRegression](https://www.kaggle.com/filipeasantos/wr-ssf-linearregression).

In [ ]:
# Prepara os dados para enviar ao R
# Inclui tamanho e tipo
trainToLinearRegression = train.merge(stores, on='Store')

# Cria a variável trend
trainToLinearRegression = trainToLinearRegression.sort_values('Date')
trainToLinearRegression['trend'] = 0
tempDate = ''
tempCount = 0
for index, row in trainToLinearRegression.iterrows():
    if row['Date'] != tempDate:
        tempDate = row['Date']
        tempCount = tempCount + 1 
    trainToLinearRegression.at[index,'trend'] = tempCount

# Remove as 3 últimas semanas para utilizar como teste
testToLinearRegression = trainToLinearRegression[trainToLinearRegression['Date'].isin(['2012-10-12','2012-10-19','2012-10-26'])]
trainToLinearRegression = trainToLinearRegression[~trainToLinearRegression['Date'].isin(['2012-10-12','2012-10-19','2012-10-26'])]

# Transforma tipo em dummies
trainToLinearRegression = pd.get_dummies(trainToLinearRegression, columns = ['Type'])

# Transforma week em dummies
trainToLinearRegression = pd.get_dummies(trainToLinearRegression, columns = ['week'])

# Transforma Dept em dummies
trainToLinearRegression = pd.get_dummies(trainToLinearRegression, columns = ['Dept'])

# Corrige as dummies (São necessárias. Para n categorias são necessárias apenas n-1 dummies)
trainToLinearRegression = trainToLinearRegression.drop(columns=['Type_C', 'week_52', 'Dept_99'])

trainToLinearRegression = trainToLinearRegression.drop(columns=['Date', 'Store', 'IsHoliday', 'StoreDept', 'year'])

print('Shape', trainToLinearRegression.shape)
print('\nTypes\n', trainToLinearRegression.dtypes)
display(trainToLinearRegression)

In [ ]:
# Salva os dados filtrados em um CSV.

#Kaggle
trainToLinearRegression.to_csv('trainToLinearRegression.csv', index=False)

#Local
#trainToLinearRegression.to_csv('data/trainToLinearRegression.csv', index=False)

In [ ]:
# Lê os resultados dos modelos de séries temporais
## Para uso no Kaggle Notebook
path = '../input/wr-ssf-linearregression/'

## Para uso local
#path = 'data/'

linearRegressionCoefficients = pd.read_csv(path + 'linearRegressionCoefficients.csv')
linearRegressionCoefficients = pd.read_csv(path + 'linearRegressionCoefficients.csv', index_col=0, header=None, squeeze=True).to_dict()
# Verifica os resultados
display(linearRegressionCoefficients)

In [ ]:
# Funcção que calcula o forecast da regressão linear
def calculeteFcstLR(size, dept, week, storeType, trend):
    if dept == 99:
        deptValue = 0
    else:
        deptValue = float(linearRegressionCoefficients['Dept_' + str(dept)])
    
    if week == 52:
        weekValue = 0
    else:
        weekValue =float(linearRegressionCoefficients['week_' + str(week)])
        
    if storeType == 'C':
        storeTypeValue = 0
    else:
        storeType = float(linearRegressionCoefficients['Type_' + str(storeType)])
    
    return  float(linearRegressionCoefficients['(Intercept)']) + deptValue + weekValue + float(linearRegressionCoefficients['trend'])*trend

# função que filtra a tabela de teste
def getRow(store, dept, date):
    row = testToLinearRegression.loc[(testToLinearRegression['Store'] == store) & (testToLinearRegression['Dept'] == dept) & (testToLinearRegression['Date'] == date)]
    if len(row) > 0:
       return row
    else:
        return 0

In [ ]:
# Calcula os forecasts para os dados de teste
# Cria um data frame para armazenar os dados
fcstsLR = fcstsETS.copy()
fcstsLR.fcst1 =	fcstsLR.fcst2 =	fcstsLR.fcst3 = pd.NA

dates = ['2012-10-12','2012-10-19','2012-10-26']
for index, row in fcstsLR.iterrows():
    for i, date in enumerate(dates):
        data = getRow(int(index.split('-')[0]),int(index.split('-')[1]),dates[i])
        size = data.Size.unique()[0]
        dept = int(index.split('-')[1])
        week = data.week.unique()[0]
        storeType = data.Type.unique()[0]
        trend = data.trend.unique()[0]
        fcstsLR.at[index,'fcst' + str(i + 1)] = calculeteFcstLR(size, dept, week, storeType, trend)


In [ ]:
# Verifica os resultados
display(fcstsLR)

# 5 Calcula o erro e compara os modelos

In [ ]:
# Cria um data frame com os dados reais
fcstTest = fcstsETS.copy()
fcstTest.fcst1 = fcstTest.fcst2 = fcstTest.fcst3 = pd.NA

dates = ['2012-10-12','2012-10-19','2012-10-26']
for index, row in fcstTest.iterrows():
    for i, date in enumerate(dates):
        data = getRow(int(index.split('-')[0]),int(index.split('-')[1]),dates[i])
        fcstTest.at[index,'fcst' + str(i + 1)] = float(data.Weekly_Sales.unique()[0])

display(fcstTest)

In [ ]:
# Calcula MAE
erroARIMA = fcstTest.subtract(fcstsARIMA).abs()
erroETS = fcstTest.subtract(fcstsETS).abs()
erroLR = fcstTest.subtract(fcstsLR).abs()

print('ARIMA\n', erroARIMA.sum()/2660)
totErroARIMA = erroARIMA.sum().sum()/(3*2660)
print(totErroARIMA)
print('\nETS\n', erroETS.sum()/2660)
totErroETS = erroETS.sum().sum()/(3*2660)
print(totErroETS)
print('\nLR\n', erroLR.sum()/2660)
totErroLR = erroLR.sum().sum()/(3*2660)
print(totErroLR)

# Compara os ARIMA e EST com LR
print('\n\nARIMA/LR', round(totErroARIMA/totErroLR,2))
print('ETS/LR', round(totErroETS/totErroLR,2))

In [ ]:
# Cria um data frame com as médias dos fcst ETS e ARIMA
fcstETSandARIMA = fcstsETS.add(fcstsARIMA)/2

erroETSandARIMA = fcstTest.subtract(fcstETSandARIMA).abs()

print('ETSandARIMA\n', erroETSandARIMA.sum()/2660)
totErroETSandARIMA = erroETSandARIMA.sum().sum()/(3*2660)
print(totErroETSandARIMA)

In [ ]:
# Compara o modelo médio com os dois modelos 
# Compara os ARIMA e EST com LR
print('\n\nETSandARIMA/ARIMA', round(totErroETSandARIMA/totErroARIMA,2))
print('ETSandARIMA/ETS', round(totErroETSandARIMA/totErroETS,2))


O modelo de regressão linear não apresentou bons resultados e precisa ser melhor trabalhado, por isso optou-se por não utiliza-lo na média. O modelo precisa ser melhor estudado e trabalhado para obter resultados melhores. Alguns estudos que podem ser feitos:
- Utilizar os demais atributos
- Verificar a remoção dos atributos utilizados
- Realizar estudos com os dados normalizados ou transformados
- Separar o modelo em categorias (por exemplo loja e departamento)
- Realizar um forecast agregado (por loja, tipo ou departemaneto) e depois desagregar utilizando uma regra proporcional
- Entender o que signific as vendas negativas em uma semana. Tentar tratar esses dados

Os dois modelos de séries temporais apresentaram resultados muito próximos e obteve-se uma ligeira melhora ao utilizar a média dos dois. A seguir listo alguns estudos que podem ser feitos para melhorar os modelos:
- Realizar a normalização ou transformação dos dados 
- Selecionar uma porcentagem dos piores modelos para gerar não utilizando os métodos automaticos
- Realizar um forecast agregado (por loja, tipo ou departemaneto) e depois desagregar utilizando uma regra proporcional
- Entender o que signific as vendas negativas em uma semana. Tentar tratar esses dados